In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install datasets transformers evaluate sentencepiece accelerate wandb ipdb


In [3]:
import pandas as pd
import os
import ipdb
import wandb


### configuration



```
# go to project path and load configuration file
```



In [4]:
project_path='/content/drive/MyDrive/multi_label_classifiction/Figurative Language of 17th Century English Plays/Figurative Language - Multil Labels Classifiction'

In [5]:
# go to project path
os.chdir(project_path)
os.getcwd()

'/content/drive/MyDrive/multi_label_classifiction/Figurative Language of 17th Century English Plays/Figurative Language - Multil Labels Classifiction'

In [6]:
#load the custom libiries of the projects
import myTrainer,myLabels,myTokenization,myEvaluation,multipleEvaluations,myDatasets

In [7]:
#read configuration and store them in config_dict
df=pd.read_csv('config.csv')
variables=df['variable_name']
chosen_values=df['chosen_option']

config_dict=dict()
for i in range(len(variables)):
  if variables[i] is not None:
    config_dict[variables[i]]=chosen_values[i]

In [8]:
#set configurations
boolean_map={'True':True,'False':False}

dataset_path=config_dict['dataset_path']
test_size=float(config_dict['test_size'])

max_labels=int(config_dict['max_labels'])
limited_labels=boolean_map[config_dict['limited_labels']]
s_idx=int(config_dict['s_idx'])
e_idx=int(config_dict['e_idx'])

finetuned_models=config_dict['finetuned_models']
models_dir=config_dict['models_dir']
model_path=config_dict['model_path']
my_wandb_API_key=config_dict['my_wandb_API_key']
kind_of_evaluation=config_dict['kind_of_evaluation']
saving_model=boolean_map[config_dict['saving_model']]
save_predictions=boolean_map[config_dict['save_predictions']]


for debuging use this commands

```
%pdb on  %pdb off
```



In [9]:
wandb.login(key=my_wandb_API_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: davidbenmi (davidbenmi-the-open-university-of-israel). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Set Datasets

In [10]:
ds=myDatasets.MyDatasets(dataset_path,test_size)
train_dataset,test_dataset=ds.get_datasets()
classes=ds.get_classes()

Filter:   0%|          | 0/1112 [00:00<?, ? examples/s]

Filter:   0%|          | 0/278 [00:00<?, ? examples/s]

### myEvaluation
choose single evaluation or multiple evaluations

In [12]:
myeval=myEvaluation.MyEvaluation(classes,model_path,max_labels,limited_labels,
                 train_dataset,test_dataset)

if (kind_of_evaluation=='single evaluation'):
  if (saving_model):
    myeval.train_on_single_ds(save_strategy="epoch",load_best_model_at_end=True,output_dir=finetuned_models,s_idx=s_idx,e_idx=e_idx)
  else:
    myeval.train_on_single_ds(s_idx=s_idx,e_idx=e_idx)

  myeval.evaluate_on_single_ds()
  myeval.print_classification_report()
  if (save_predictions):
    myeval.save_predictions()

if(kind_of_evaluation=='multiple evaluations'):
  multiEval=multipleEvaluations.MultipleEvaluations(myeval,classes,max_labels,limited_labels,s_idx=s_idx,e_idx=e_idx,
                                                    saving_model=saving_model,output_dir=finetuned_models)
  multiEval.set_evaluated_models_path(models_dir,project_path)
  multiEval.evaluateModels(step=10)
  multiEval.print_classifiction_report()
  if (save_predictions):
    multiEval.save_predictions_in_csv_file()

Map:   0%|          | 0/1111 [00:00<?, ? examples/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1111 [00:00<?, ? examples/s]

Filter:   0%|          | 0/278 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/content/drive/MyDrive/multi_label_classifiction/Figurative Language of 17th Century English Plays/Figurative Language - Multil Labels Classifiction/myTrainer.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.189538
2,No log,0.169505
3,No log,0.147476
4,No log,0.147094


                                 precision    recall  f1-score   support

                        anatomy       0.97      1.00      0.99        74
                       violence       0.91      0.91      0.91        54
                       politics       0.97      0.89      0.93        37
                       emotions       0.97      0.89      0.93        44
                       religion       0.97      0.95      0.96        37
mental_faculty__state__entities       1.00      0.95      0.97        37
                         nature       1.00      0.91      0.96        35
                        spatial       0.95      0.88      0.91        24
                      economics       0.93      0.86      0.89        29
       character_traits__nature       0.95      0.63      0.76        30
                           food       1.00      0.88      0.94        26
                       language       1.00      0.88      0.93        24
                           time       0.95      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
